## Get and pre-process all data

In [111]:
import requests
# get abr entry
import requests as r

t_id = 4541
url = f"https://alwaysberunning.net/api/entries?id={t_id}"
tourney = requests.get(url).json()

abr_names = [""] * len(tourney)
for i, entry in enumerate(tourney):
    if entry["user_import_name"] is not None and entry["rank_top"] is not None:
        abr_names[i] = (entry["user_import_name"], entry["rank_top"])
    elif entry["user_import_name"] is not None and entry["rank_top"] is None:
        abr_names[i] = (entry["user_import_name"], entry["rank_swiss"])
    elif entry["rank_top"] is not None:
        abr_names[i] = (entry["user_name"], entry["rank_top"])
    else:
        abr_names[i] = (entry["user_name"], entry["rank_swiss"])


abr_names.sort(key=lambda x: x[1])
abr_names = [x[0] for x in abr_names]

 # special chars are baaaaad
if t_id == 4541: abr_names[43] = "l0velace"


abr_names_set = set(abr_names)
# abr_names_set

In [21]:
import pandas as pd

In [113]:
col_names = {
    "Basic Info": "name",
    "Email": "email",
    "Phone Number": "phone",
    "Shipping address (Please make sure this is your entire address including country 🙏❤️ )": "shipping",
    "IMPORTANT* Name/Handle you will use on Cobra so I can connect your placement to your prizes.": "handle",
    "Participation Prizes choice #1 (playset)": "part_one",
    "Participation Prizes choice #2 (playset)": "part_two",
    "Additional prizing 🥳 (check all that apply)": "add_prizing",
    "Do you already have a CBI invite (important)? Do you want the playmat?": "cbi",
    "VIP": "playmat",
    "Delivery options. Is your shipping address correct? (includes a country and correct formatting)": "shipping_options"
}


df = pd.read_csv('data/data.tsv', sep='\t')

df.rename(columns=col_names, inplace=True)
df = df[['name', 'email', 'phone', 'shipping', 'handle', 'part_one', 'part_two', 'add_prizing', 'cbi', 'shipping_options', 'playmat']]

df['placement'] = None

print(df.head())

   name  email  phone  shipping             handle             part_one  \
0   NaN    NaN    NaN       NaN  marasmia (xe/xem)  Finality (Cat Shen)   
1   NaN    NaN    NaN       NaN           MattOhNo  Finality (Cat Shen)   
2   NaN    NaN    NaN       NaN          Porkobolo  Finality (Cat Shen)   
3   NaN    NaN    NaN       NaN        awildturtok    Revolver (PreNic)   
4   NaN    NaN    NaN       NaN      CodeMarvelous  Finality (Cat Shen)   

                                            part_two  \
0  Sure Gamble (Cassius Marcellus Coolidge &amp; ...   
1                     Full art Slap Vandal (NSG GNK)   
2  Sure Gamble (Cassius Marcellus Coolidge &amp; ...   
3                                Finality (Cat Shen)   
4                                  Revolver (PreNic)   

                                         add_prizing  \
0                                            "🎩 🧢 👒"   
1                                                NaN   
2                                           

In [130]:
# process
# enter player aliases here

correct_handles = {
    "Lastwarrior/AbyssStaresBack": "AbyssStaresBack",
    "marasmia (xe/xem)": "marasmia",
    "Max/ChonkySeal": "ChonkySeal",
    "PiCat": "PiCat314",

}

rem_players = abr_names_set.copy()

for player in df.itertuples():
    if pd.isnull(player.handle): break
    if player.handle in correct_handles.keys():
        df.at[player.Index, "handle"] = correct_handles[player.handle]
    handle = player.handle

    if handle in rem_players:
        df.at[player.Index, "placement"] = abr_names.index(handle)
        rem_players.remove(handle)
        continue

    print(handle)
print(rem_players)

Lastwarrior/AbyssStaresBack
Max/ChonkySeal
marasmia (xe/xem)
PiCat
cody1024d
CyberShark
Baxder
ThatRedHyena
grundmc88
bdorf
amxrt
TheSovietLoveHammer
aashbo
Nut_Milk
kevinth
theRemix
{'AbyssStaresBack', 'ArminFirecracker', 'IAmFrog', 'xou', 'Psygamer', 'benvelopment', 'Baa Ram Wu', 'ChonkySeal', 'Toxiccord', 'Cise Starrgate', 'CobraBubbles', 'Larrea', 'marasmia', 'Brita', 'rubenpieters', 'ExperimentalDataCore', 'pink opaque', 'Baserton', 'TorpedoTyrus', 'TannenLimoLiebe', 'PiCat314', 'WholockianDalek', 'OF15-15'}


In [131]:
df = df.sort_values(by="placement")
print(df[["handle", "placement"]])

                 handle placement
53                 ant1         0
29             not_yeti         2
67      AbyssStaresBack         3
25                Ollie         5
26          AnOddRadish         6
..                  ...       ...
62  TheSovietLoveHammer      None
71               aashbo      None
85             Nut_Milk      None
89              kevinth      None
90             theRemix      None

[92 rows x 2 columns]


In [133]:
playmat = 8
cbi = 5


playmats = list(df[~df['cbi'].str.contains('I already won a playmat')].head(playmat)["handle"])
no_invites = list(df[df['cbi'].str.contains('I already won a CBI invite.')]["handle"])
tickets = list(df["handle"])


print(f"playmat to \n {playmats}\n\n")
print(f"ask for invite: {[i for i in abr_names[:abr_names.index(invites[-1])] if i not in tickets]}\n")
print(f"potential invites: {[i for i in abr_names[:abr_names.index(invites[-1])] if i not in no_invites]}\n")


playmat to 
 ['ant1', 'not_yeti', 'AbyssStaresBack', 'Ollie', 'AnOddRadish', 'davz131', 'Buachu', 'tzeentchling']


ask for invite: ['OF15-15', 'CobraBubbles', 'TorpedoTyrus']

potential invites: ['ant1', 'OF15-15', 'CobraBubbles', 'Buachu', 'TorpedoTyrus', 'Ish', 'cat']

